# Overview of the datacube

This notebook handles download of a pre-made datacube, and some visualisations to help understanding of it and check its health.

In [ ]:
import os
import pooch
import pandas as pd
import numpy as np
import xarray as xr
import dask as da
import zarr
import hvplot.xarray

# # from dask.distributed import Client
# # from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.cm as cm
# from matplotlib.colors import Normalize
# from matplotlib.colorbar import ColorbarBase

from chaosmagpy.plot_utils import nio_colormap

try:
    from src.data.proc_env import INT_FILE_PATHS
    from src.env import REFRAD, TMPDIR
except ImportError:
    print("Failed to import src...")
    TMPDIR = os.getcwd()
    INT_FILE_PATHS = {"A": os.path.join(TMPDIR, "SwA_20140501-20190501_proc1.nc")}
    REFRAD = 6371200
    print("Using instead for cube download and scratch space:", TMPDIR)
if not os.path.exists(TMPDIR):
    print("Can't find scratch space:", TMPDIR)
    TMPDIR = os.getcwd()
    print("Using instead:", TMPDIR)

In [ ]:
print(
    "Using temporary working directory:",
    TMPDIR,
    "Is this a good location for data I/O? Configure this path in the file: geomagnetic_datacubes_dev/config.ini",
    "(or manually enter new paths above if not using the geomagcubes environment)",
    sep="\n"
)

## Load/prepare datacube

### Download pre-made datacube

This part to be refactored into the datacube generation pipeline, when a permanent link is made available.

Download the file if we don't already have it available here.

In [ ]:
# The location at which the data will be located
fpath = INT_FILE_PATHS["A"]
path, fname = os.path.split(fpath)
path, fname

In [ ]:
# # Delete it if you want to redownload it
# os.remove(fpath)

In [ ]:
if os.path.exists(fpath):
    # Skip the download if we already have the file
    print("Already found file:", fpath, sep="\n")
    pass
else:
    pooch.retrieve(
        url="https://nc.smithara.net/index.php/s/H5R923DsbtirfJy/download",
        known_hash="1b7a8cbc0cb1657f8d4444ae7f6bbab91841318e1a172fa1f8a487b9d9492912",
        path=path, fname=fname,
        progressbar=True,
    )

### Make a copy of the input datacube as a Zarr store

We want to make sure we don't accidentally modify the input dataset, so let's work on a copy. There are also some opportunities with xarray and dask and the zarr format to increase performance by dividing into chunks / rearranging the data in different ways - the input data format is not necessarily what we want to use for computation. So here we convert the data to the [Zarr](https://zarr.readthedocs.io) format

(could work with the .nc file just the same; not sure yet what the advantages of zarr are)

In [ ]:
file_in = INT_FILE_PATHS["A"]
zarr_store = os.path.join(TMPDIR, "datacube_test.zarr")
print("Input file:", file_in, "Copying to:", zarr_store, sep="\n")

In [ ]:
if os.path.exists(zarr_store):
    print("Already found zarr:", zarr_store, sep="\n")
    pass
else:
    with xr.open_dataset(file_in) as ds:
        ds.to_zarr(zarr_store)

## Diagnostics of data

In [ ]:
ds = xr.open_dataset(
    zarr_store, engine="zarr",
    chunks="auto"
)
# Remove the sources for now (to be fixed later)
ds.attrs.pop("Sources")
ds

Assuming input 1Hz data, this is how much the data has been decimated by  
(i.e. it is 10s sampling, with a bit more lost due to quality Flags)

In [ ]:
timedelta_ns = float(ds["Timestamp"].isel(Timestamp=-1) - ds["Timestamp"].isel(Timestamp=0))
print("Fraction of input data:", len(ds["Timestamp"]) / (timedelta_ns/1e9))

### Spatial variation of magnetic field data, and data-model residuals

Do some tricks to generate manageable summary images...

First downsample again so we don't needlessly work with all the data just for these visualisations:

In [ ]:
# Dataset downsampled by 1/30 (i.e. 5-minute sampling)
_ds = ds.isel(Timestamp=slice(0, -1, 30))
# Generate residuals to plot
_ds["B_NEC_res_CHAOS-full"] = (
    _ds["B_NEC"]
    - _ds["B_NEC_CHAOS-MCO"]
    - _ds["B_NEC_CHAOS-MMA"]
    - _ds["B_NEC_CHAOS-Static_n16plus"]
)

These next plots use `hvplot` (using `holoviews`) underneath to generate interactive `bokeh` plots - this is quite tricky to work with so better left alone until you have mastered `matplotlib`.

In [ ]:
def plot_NEC_var(_ds=_ds, var="B_NEC", **kwargs):
    return (
        _ds.drop("Timestamp")
        .hvplot.scatter(
            x="Longitude", y="Latitude", c=var,
            by="NEC", subplots=True,
            rasterize=True,
            colorbar=True,
            hover=True,
            width=300, height=200,
            **kwargs
        )
    )


print("B_NEC: magnetic field measurements")
plot_NEC_var(_ds=_ds, var="B_NEC", clim=(-50000, 50000), cmap=nio_colormap())

In [ ]:
print("B_NEC_res_CHAOS-full: The effect of removing the full CHAOS model, comprising core, magnetosphere, and lithosphere. i.e. mostly space weather signals remaining")
plot_NEC_var(_ds, "B_NEC_res_CHAOS-full", clim=(-50, 50), cmap=nio_colormap())

In [ ]:
# Masks to use for data subselection
nightside = (_ds["SunZenithAngle"] > 100)
nightside_quiet = nightside & (_ds["Kp"] < 3)
nightside_quiet_low_MEF = nightside_quiet & (_ds["IMF_Em"] < 0.8)

In [ ]:
def _plot_stdvs(_ds, ax, title):
    (
        _ds
       .groupby_bins("QDLat", 90)
       .std()["B_NEC_res_CHAOS-full"]
       .plot.line(x="QDLat_bins", ax=ax)
    )
    ax.set_title(title)
    ax.set_ylabel("")


fig, axes = plt.subplots(ncols=4, figsize=(20, 5), sharey=True, sharex=True)
_plot_stdvs(_ds, axes[0], "Without data selection")
_plot_stdvs(_ds.where(nightside), axes[1], "Night")
_plot_stdvs(_ds.where(nightside_quiet), axes[2], "Night; Kp<3")
_plot_stdvs(_ds.where(nightside_quiet_low_MEF), axes[3], "Night; Kp<3; $E_m$<0.8")
axes[0].set_ylim((0, 200))
axes[0].set_ylabel("B_NEC_res, standard deviations [nT]");

Above: the spread of residuals found under increasingly stringent data selection; i.e. why we typically use geomagnetically quiet nightside data for internal field modelling. For a deeper dive on this, see https://swarm.magneticearth.org/notebooks/04a1_geomag-models-vires

It seems there are some data spikes to be removed still - ideally these should be caught be the quality flags.  
(TODO: Remove these from the datacube)

In [ ]:
_ds["B_NEC_res_CHAOS-full"].max().compute()

In [ ]:
# Scrub them out with something like this
outliers = np.fabs(_ds["B_NEC_res_CHAOS-full"]) > 2000
_ds.where(~outliers)["B_NEC_res_CHAOS-full"].max().compute()

### Begin exploring relationships between parameters...

In [ ]:
north_auroral_oval = (_ds["QDLat"] > 60) & (_ds["QDLat"] < 80)
(
    _ds.where(north_auroral_oval & ~outliers, drop=True)
    .drop("Timestamp")
    .sel(NEC="C")
    .plot.scatter(
        x="IMF_Em", y="B_NEC_res_CHAOS-full", s=1
    )
)

It is possible to find correlations between the residuals and solar wind parameters such as merging electric field (`IMF_Em` in the datacube; sometimes referred to as $E_m$). This needs to be explored also as a function of position within QDLat / MLT. See Figure 8.1 (page 135) in my thesis (https://doi.org/10.5281/zenodo.3952719)

### Temporal information

In [ ]:
_ds["Altitude"] = (_ds["Radius"] - REFRAD)/1e3
_ds["Altitude"].attrs = {"units": "km"}
_ds["Altitude"].plot.line(x="Timestamp");